In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [4]:
df = df[['Title','Plot','Genre']]

In [5]:
df.head()

,Title,Plot,Genre
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",unknown
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",unknown
2,The Martyred Presidents,"The film, just over a minute long, is composed...",unknown
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,unknown
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...,unknown


In [6]:
genres = df.Genre.unique()
print(genres)
genres.size

['unknown' 'western' 'comedy' ... 'adventure, romance, fantasy film' 'ero'
 'horror romantic comedy']


2265

In [7]:
df = df[df.Genre!='unknown']
df = df[~df.Genre.str.contains(r",|/| |-", regex=True)]
df.head()

,Title,Plot,Genre
6,The Great Train Robbery,The film opens with two bandits breaking into ...,western
7,The Suburbanite,The film is about a family who move to the sub...,comedy
10,Dream of a Rarebit Fiend,The Rarebit Fiend gorges on Welsh rarebit at a...,short
13,Daniel Boone,Boone's daughter befriends an Indian maiden as...,biographical
14,How Brown Saw the Baseball Game,Before heading out to a baseball game at a nea...,comedy


In [8]:
df.count

<bound method DataFrame.count of                                  Title  \
6              The Great Train Robbery   
7                      The Suburbanite   
10            Dream of a Rarebit Fiend   
13                        Daniel Boone   
14     How Brown Saw the Baseball Game   
...                                ...   
34875     Once Upon a Time in Anatolia   
34876                            Selam   
34882               Çalgı Çengi İkimiz   
34883                     Olanlar Oldu   
34885               İstanbul Kırmızısı   

                                                    Plot         Genre  
6      The film opens with two bandits breaking into ...       western  
7      The film is about a family who move to the sub...        comedy  
10     The Rarebit Fiend gorges on Welsh rarebit at a...         short  
13     Boone's daughter befriends an Indian maiden as...  biographical  
14     Before heading out to a baseball game at a nea...        comedy  
...                     

In [9]:
df.Genre.value_counts().head(10)

Genre
drama        5964
comedy       4379
horror       1167
action       1098
thriller      966
romance       923
western       865
crime         568
adventure     526
musical       467
Name: count, dtype: int64

In [10]:
valid_genres = df['Genre'].value_counts()[df['Genre'].value_counts() >=923].index
df = df[df['Genre'].isin(valid_genres)]
df = df.groupby("Genre").sample(n = 923,random_state=69)
df.head()

,Title,Plot,Genre
13517,The Mummy,"In Thebes, Egypt, 1290 BC, high priest Imhotep...",action
13660,Gun Shy,Charlie Mayeaux (Liam Neeson) is an undercover...,action
22102,American Soldiers,"Iraq, 2004: during a routine sortie a US patro...",action
16732,"I, Frankenstein","In 1795, Dr. Victor Frankenstein (Aden Young) ...",action
11421,Hard to Kill,"In 1983, Mason Storm, a Los Angeles police det...",action


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [ ]:
X = df['Plot']
y = df['Genre']
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=69)
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2),stop_words="english",max_features=20000)),
    ("clf", LogisticRegression(max_iter=200, C=2.0, solver='liblinear'))
])
pipeline.fit(X_train,Y_train)
y_pred = pipeline.predict(X_test)
scores = cross_val_score(pipeline, X, y, cv=5)

print("Scores for each fold:", scores)
print("Average accuracy:", scores.mean())

Scores for each fold: [0.55685921 0.5631769  0.56227437 0.53206865 0.58717254]
Average accuracy: 0.5603103323452008
